In [ ]:
###
#This script is to scrape the times that a participant OPENED an EMA survey
#The Ethica data export only includes the time that a participant completes a survey, which may not be useful
#We are particularly interested in when a participant STARTED EMA, resulting in a need to grab timestamps
###

#selenium for being able to browse the site while loading the JavaScript it uses
#pandas for putting the scraped data into a data frame

import selenium
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import numpy as np
import pandas as pd

In [ ]:
#Getting a browser that can get to Ethica's JavaScript page
#You will need to download the driver at https://selenium-python.readthedocs.io/installation.html#drivers
#Once downloaded, extract the file and save the .exe in your file path for python
#You can find the path using:

#import os
#import sys
#os.path.dirname(sys.executable)
    
driver = webdriver.Chrome()

driver.get("http://www.google.com/")
time.sleep(1)
#chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument("--incognito")

#driver.get('https://google.com')

In [ ]:
driver.get("https://ethicadata.com/app/dashboard/auth/sign-in")


#For some reason, on occasion this won't log in when running the code straight up, but when running this cell again it works...
#Remember to input your username and password in this cell - I've removed the lab's credentials for obvious security reasons
driver.find_element_by_name("username").send_keys("EMAIL@WHATEVER.COM")
driver.find_element_by_name("password").send_keys("PUTYOURPASSWORDHERE")
driver.find_element_by_class_name("jss48").click()

In [ ]:
#This cell pulls a CSV that contains our participant's study IDs and their respective Ethica ID assigned by Ethica
#Need to make sure the file path matches where the "Ethica IDs.csv" file is

ids_df = pd.read_csv(r"C:\Users\Luis\Box\R15 Sensor Preprocessing and Analysis\Data\Ethica IDs.csv", index_col = [0])
ids_df

In [ ]:
#Could probably automate this...
#Input the participant's ID that you're interested in and it will read the prior .csv to grab their Ethica ID
print("Please type in the study ID for this participant.")
study_id = input()

In [ ]:
#Turning the ID into a string
ethica_id = ids_df.loc[study_id].values[0]
ethica_id = int(ethica_id)
ethica_id = str(ethica_id)
ethica_id

In [ ]:
#URL for getting participants timestamps for opening the survey, eventIDs=200 is events where a participant opened a survey
driver.get("https://ethicadata.com/app/dashboard/rdash/study/1287/phr/?dateFrom&dateTo&eventIds=200&participantIds=" + ethica_id)
time.sleep(3)

In [ ]:
#grabbing web elements for the next part
timestamps = driver.find_elements_by_xpath('//*[@id="root"]/div/div[3]/div[2]/div/div/div[2]/div[2]/div/div/div/div/div[2]/div/*/div/div[1]')
ethScroll = driver.find_element_by_xpath('//*[@id="root"]/div/div[3]/div[2]/div/div/div[2]/div[2]/div/div/div/div/div[2]')


In [ ]:
#This portion of code is what's pulling the timestamps.
#It's a little slow because it needs to retain the elements in memory
#1 second sleep was too low, 2 second sleep seems to work. Could probably test and reduce. 
#I could probably also make it automatically stop once it's reached the bottom of the page, but alas

timestamp_text = []

for x in range(0,20):
    timestamps = driver.find_elements_by_xpath('//*[@id="root"]/div/div[3]/div[2]/div/div/div[2]/div[2]/div/div/div/div/div[2]/div/*/div/div[1]')
    time.sleep(2)
    for stamps in timestamps:
        timestamp_text.append(stamps.text)
    driver.execute_script("arguments[0].scrollBy(0, 500)", ethScroll)



In [ ]:
#Just creating a data frame here and removing duplicate cases
#tbh could probably simplify this into one line 
timestamp_df = pd.DataFrame(np.array(timestamp_text))
timestamp_df = timestamp_df.drop_duplicates()

timestamp_df

In [ ]:
#timestamp_df.to_csv('C:\Users\astech\Empatica Data\ID PR*\timestamps.csv', index=False)
#Saving to a CSV - I definitely re-wrote this line to pull the ID from the study_id input but apparently not on this notebook.

timestamp_df.to_csv('C:/Users/Luis/Desktop/ID PR###/timestamps.csv', index=False)